version - 0.1.9.8 (240304)
Reordered the takesByLine dictionary for ease of use in jsx

##### Import

In [1]:
%%html
<style>
table.dataframe th {text-align: center;}
table.dataframe td {text-align: left;}
</style>
from IPython.core.display import HTML


HTML("""
<style>
.dataframe td, .dataframe th {
    max-width: None;
    overflow: hidden;
    text-overflow: ellipsis;
    white-space: normal;
    table.dataframe th {
        text-align: center;
    }
    table.dataframe td {
        text-align: left;
    }
}
</style>
""")

import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [2]:
""" ## Imports """
import nltk
import pprint as pp
import re
import json
import numpy as np
import torch
import os
from IPython.display import Markdown, display
import math
import matplotlib.pyplot as plt
import seaborn as sns
import csv
import tqdm
import pandas as pd

# Parsing Logic
from Levenshtein import distance
from sklearn.metrics.pairwise import cosine_similarity
from transformers import BertTokenizer, BertModel
from sentence_transformers import SentenceTransformer, util
from InstructorEmbedding import INSTRUCTOR

# Skipping In a For Loop
from itertools import islice
import collections

# Whisper Transcription
import whisper_timestamped as whisper
import datetime

def display_df(df):
    display(df.style.set_properties(**{'text-align': 'left'}).set_table_styles([dict(selector="th", props=[('text-align', 'center')])]))

def print_bold(text):
    display(Markdown('**' + text + '**'))

def pprint(obj, width=100):
    return pp.pprint(obj, width=width, compact=True, sort_dicts=False, underscore_numbers=True)

def printright(string, value, padding=30):
    x = string.rjust(padding, ' ')
    print(f'{x} {value}')
    
def spacer(i):
    hype = '-' * (i + 1)
    printright(hype, value='', padding=30)
    
def dateTime(file_name):
    now = datetime.datetime.now()
    date_time_str = now.strftime("%y%m%d-%H%M")    
    return f"{file_name}_{date_time_str}"



Importing the dtw module. When using in academic works please cite:
  T. Giorgino. Computing and Visualizing Dynamic Time Warping Alignments in R: The dtw Package.
  J. Stat. Soft., doi:10.18637/jss.v031.i07.



/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/whisper/timing.py:57: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit


In [7]:
""" ## Word-Level Transcription """
def wlTranscribe (file_path):
    audio = whisper.load_audio(file_path)
    model = whisper.load_model("medium", device="cpu")
    trans_base_dict = whisper.transcribe(model, audio, language="en", beam_size=5, best_of=5, temperature=(0.0, 0.2, 0.4, 0.6, 0.8, 1.0), vad="auditok",verbose=True)
    
    return trans_base_dict

from deepmultilingualpunctuation import PunctuationModel
def addPunct(text):
    model = PunctuationModel()
    result = model.restore_punctuation(text)
    return result

def toJson(dictionary, file_name, prefix=""):
    if prefix != "":
        file_name_with_prefix = f"{prefix}_{file_name}"
        file_name = file_name_with_prefix
    file_name_with_date = dateTime(file_name)
    file_name_with_date += ".json"

    with open(file_name_with_date, "w") as file:
        json.dump(dictionary, file, indent=4)

def process_folder(folder_path):
    for file in os.listdir(folder_path):
        if file.endswith(".m4a"):  # Fix: Pass file types as a tuple
            file_path = os.path.join(folder_path, file)
            file_name = os.path.splitext(file)[0]

            print(f"Processing file: {file}")
            result = wlTranscribe(file_path)
            toJson(result, file_name, prefix='ASR')


# Specify the folder containing the audio files
audio_folder = "/Users/tristangardner/Documents/Suora/Suora Course/Mod 5 Dictation"
transcribe = True
if transcribe == True: 
    process_folder("/Users/tristangardner/Documents/Suora/Suora Course/Mod 5 Dictation")

Processing file: Mod 5 Draft 2.m4a


 12%|█▏        | 5222/42681 [00:55<06:45, 92.47frames/s] 

: 

: 

: 

In [8]:
""" ## Pre-Proccess Scripts """
def preprocess_script(file_path):
    with open(file_path, 'r') as file:
        text = file.read()
    
    text_start = text.find('"')
    text_end = text.rfind('"')    
    text = text[text_start+2:text_end-1]
    
    # Text to process
    # Mapping of numbers to words
    num_to_word = {'1': 'One', '2': 'Two', '3': 'Three', '4': 'Four', '5': 'Five', '6': 'Six', '7': 'Seven', '8': 'Eight', '9': 'Nine'}

    # Replace single-digit numbers followed by a period with words
    text = re.sub(r'\b([1-9])\.', lambda x: num_to_word[x.group(1)] + ',', text)
    text = re.sub(r'[;:]', '.', text) # Replace semi-colons and colons with periods. 

            
    # Tokenize the text into sentences
    sentences = nltk.sent_tokenize(text)
    #preprocessed_sentences = [re.sub(r'[^-\w \']', '', sentence).lower() for sentence in sentences]
    #preprocessed_sentences = [re.sub(r'[\n\']', '', sentence) for sentence in sentences]
    preprocessed_sentences = [re.sub(r'[\u2022\t]', ' ', sentence) for sentence in sentences] # Remove bullet points
    preprocessed_sentences = [re.sub(r"\[.*?\]", '', sentence) for sentence in preprocessed_sentences] # Remove html tags

    preprocessed_sentences = [re.sub(r'[\n]', '', sentence).strip() for sentence in preprocessed_sentences] # Remove tabs

    
    for index, sentence in enumerate(preprocessed_sentences):
        if index > 0:
            if sentence[-1:] in ["'",'"']:
                preprocessed_sentences[index-1] = preprocessed_sentences[index-1] + ' ' + sentence
                preprocessed_sentences[index] = None
                
    preprocessed_sentences = [sentence for sentence in preprocessed_sentences if sentence is not None]    
    return preprocessed_sentences

def preProcessScriptFolder(script_folder):
    for file in os.listdir(scripts_folder):
        base_path = scripts_folder + '/'
        filepath = base_path + file
        processed_script = preprocess_script(filepath)
        
        script_title = os.path.splitext(file)[0]
        with open(filepath, 'w') as file:
            file.write(script_title +'\n"\n')
            for sentence in processed_script:
                file.write(sentence + '\n')
            file.write('"')
            
# scripts_folder = '/Users/tristangardner/Documents/Programming/3. Test Media/Wayne Mayer/Scripts'


In [14]:
""" ## Main Emeddings & Cutting Functions """
def get_bert_embeddings(string_sequence):
    # Load pre-trained BERT model and tokenizer
    model_name = 'bert-base-cased'           #'bert-base-uncased'
    tokenizer = BertTokenizer.from_pretrained(model_name)
    model = BertModel.from_pretrained(model_name)

    # Tokenize and encode the sequence
    tokens = tokenizer(string_sequence, return_tensors='pt', padding=True, truncation=True)

    # Get the BERT embeddings for the sequence
    with torch.no_grad():
        embeddings = model(**tokens).last_hidden_state.mean(dim=1)

    return embeddings

def getCuts(input_dictionary):

    df = pd.DataFrame(input_dictionary)

    output_dictionary = {
        'cut id': [],
        'start/end': [],
        'sentence indices': [],
        'sentences': [],
        'group ids': [],
        'similarity': [],
        'take indices': [],
        'asr matches': []
    }

    new_row = 0
    cut_counter = 0
    for i in range(len(df)):
        if i == 0 or df.loc[i, 'Group ID'] != df.loc[i - 1, 'Group ID']:
            new_row = i  # Marks the start of a new group

        # When we get to the last row in a group or the end of the dataframe
        if i == len(df) - 1 or df.loc[i, 'Group ID'] != df.loc[i + 1, 'Group ID']:
            # Define the end of the group
            group_end = i

            # When at the end of a group, assign start and end times as a tuple
            group_start_time = df.at[new_row, 'Start Time']
            group_end_time = df.at[group_end, 'End Time']

            # Collect all unique script sentences, take indices, and ASR matches in the group
            unique_script_sentences = df.loc[new_row:group_end, 'Script Sentence'].unique()
            take_indices = df.loc[new_row:group_end, 'Take Index'].tolist()
            asr_matches = df.loc[new_row:group_end, 'ASR Matches'].tolist()
            group_similarity = df.loc[new_row:group_end, 'Similarity Score'].mean()
            group_id = df.at[new_row, 'Group ID']

            # Increment the counter and append the values to the dictionary
            cut_counter += 1
            output_dictionary['cut id'].append(cut_counter)
            output_dictionary['start/end'].append((group_start_time, group_end_time))
            output_dictionary['sentence indices'].append(df.at[new_row, 'Script Sentence Index'])
            output_dictionary['sentences'].append(' '.join(unique_script_sentences))
            output_dictionary['group ids'].append(group_id)
            output_dictionary['similarity'].append(group_similarity)
            output_dictionary['take indices'].append(take_indices)
            output_dictionary['asr matches'].append(asr_matches)

    return output_dictionary


#! testing reorder 240304
def getCutsByGroupAndSentence(input_dictionary):
    df = pd.DataFrame(input_dictionary)

    output_dictionary = {}

    cut_counter = 0
    # Group by both 'Script Sentence Index' and 'Group ID'
    grouped_df = df.groupby(["Script Sentence Index", "Group ID"])

    for (script_sentence_index, group_id), group in grouped_df:
        # Get the start and end time for the group
        group_start = group["Start Time"].min()
        group_end = group["End Time"].max()
        similarity = group["Similarity Score"].mean()

        # Collect all unique script sentences in the group
        unique_script_sentences = group["Script Sentence"].unique()
        combined_sentences = " ".join(unique_script_sentences)

        # Collect the take indices and their corresponding ASR matches
        take_indices = group["Take Index"].tolist()
        asr_matches = group["ASR Matches"].tolist()

        # Increment the counter and append the values to the dictionary
        cut_counter += 1
        cut_info = {
            "cut id": cut_counter,
            "start/end": (group_start, group_end),
            "sentences": combined_sentences,
            "group ids": group_id,
            "similarity": similarity,
            "take indices": take_indices,
            "asr matches": asr_matches,
        }

        # Convert script_sentence_index to string and add the cut info to the output dictionary, nested under the sentence index
        script_sentence_index = str(script_sentence_index)
        if script_sentence_index not in output_dictionary:
            output_dictionary[script_sentence_index] = []
        output_dictionary[script_sentence_index].append(cut_info)

    # Sort the cuts within each sentence index group by similarity, from largest to smallest
    for sentence_index, cuts in output_dictionary.items():
        output_dictionary[sentence_index] = sorted(
            cuts, key=lambda x: x["similarity"], reverse=True
        )

    return output_dictionary


#! save 240204
""" 
def AAAAgetCutsByGroupAndSentence(input_dictionary):
    # df = pd.DataFrame(input_dictionary)

    # output_dictionary = {
    #     'cut id': [],
    #     'start/end': [],
    #     'sentence indices': [],
    #     'sentences': [],
    #     'group ids': [],
    #     'similarity': [],
    #     'take indices': [],
    #     'asr matches': []
    # }

    # cut_counter = 0
    # # Group by both 'Script Sentence Index' and 'Group ID'
    # grouped_df = df.groupby(['Script Sentence Index', 'Group ID'])

    # for (script_sentence_index, group_id), group in grouped_df:
    #     # Get the start and end time for the group
    #     group_start = group['Start Time'].min()
    #     group_end = group['End Time'].max()
    #     similarity = group['Similarity Score'].mean()

    #     # Collect all unique script sentences in the group
    #     unique_script_sentences = group['Script Sentence'].unique()
    #     combined_sentences = ' '.join(unique_script_sentences)

    #     # Collect the take indices and their corresponding ASR matches
    #     take_indices = group['Take Index'].tolist()
    #     asr_matches = group['ASR Matches'].tolist()

    #     # Increment the counter and append the values to the dictionary
    #     cut_counter += 1
    #     output_dictionary['cut id'].append(cut_counter)
    #     output_dictionary['start/end'].append((group_start, group_end))
    #     output_dictionary['sentence indices'].append(script_sentence_index)
    #     output_dictionary['sentences'].append(combined_sentences)
    #     output_dictionary['group ids'].append(group_id)
    #     output_dictionary['similarity'].append(similarity)
    #     output_dictionary['take indices'].append(take_indices)
    #     output_dictionary['asr matches'].append(asr_matches)

    return output_dictionary
 """

" \ndef AAAAgetCutsByGroupAndSentence(input_dictionary):\n    # df = pd.DataFrame(input_dictionary)\n\n    # output_dictionary = {\n    #     'cut id': [],\n    #     'start/end': [],\n    #     'sentence indices': [],\n    #     'sentences': [],\n    #     'group ids': [],\n    #     'similarity': [],\n    #     'take indices': [],\n    #     'asr matches': []\n    # }\n\n    # cut_counter = 0\n    # # Group by both 'Script Sentence Index' and 'Group ID'\n    # grouped_df = df.groupby(['Script Sentence Index', 'Group ID'])\n\n    # for (script_sentence_index, group_id), group in grouped_df:\n    #     # Get the start and end time for the group\n    #     group_start = group['Start Time'].min()\n    #     group_end = group['End Time'].max()\n    #     similarity = group['Similarity Score'].mean()\n\n    #     # Collect all unique script sentences in the group\n    #     unique_script_sentences = group['Script Sentence'].unique()\n    #     combined_sentences = ' '.join(unique_script_se

In [ ]:
def getCutsByGroupAndSentence(input_dictionary):
    df = pd.DataFrame(input_dictionary)

    output_dictionary = {}

    cut_counter = 0
    # Group by both 'Script Sentence Index' and 'Group ID'
    grouped_df = df.groupby(["Script Sentence Index", "Group ID"])

    for (script_sentence_index, group_id), group in grouped_df:
        # Get the start and end time for the group
        group_start = group["Start Time"].min()
        group_end = group["End Time"].max()
        similarity = group["Similarity Score"].mean()

        # Collect all unique script sentences in the group
        unique_script_sentences = group["Script Sentence"].unique()
        combined_sentences = " ".join(unique_script_sentences)

        # Collect the take indices and their corresponding ASR matches
        take_indices = group["Take Index"].tolist()
        asr_matches = group["ASR Matches"].tolist()

        # Increment the counter and append the values to the dictionary
        cut_counter += 1
        cut_info = {
            "cut id": cut_counter,
            "start/end": (group_start, group_end),
            "sentences": combined_sentences,
            "group ids": group_id,
            "similarity": similarity,
            "take indices": take_indices,
            "asr matches": asr_matches,
        }

        # Convert script_sentence_index to string and add the cut info to the output dictionary, nested under the sentence index
        script_sentence_index = str(script_sentence_index)
        if script_sentence_index not in output_dictionary:
            output_dictionary[script_sentence_index] = []
        output_dictionary[script_sentence_index].append(cut_info)

    # Sort the cuts within each sentence index group by similarity, from largest to smallest
    for sentence_index, cuts in output_dictionary.items():
        output_dictionary[sentence_index] = sorted(
            cuts, key=lambda x: x["similarity"], reverse=True
        )

    return output_dictionary

### Main Logic

In [23]:
def renameFile(folder_path, old_string, new_string):
    import os
    for file in os.listdir(folder_path):
        if old_string in file:
            old_file = os.path.join(folder_path, file)
            new_file = old_file.replace(old_string, new_string)
            updated_file = os.path.join(folder_path, new_file)
            os.rename(old_file, updated_file)

folder_path = ''


**FILES**

In [33]:
""" ## (1) Input Files, sort and group ASR files by script """
##! asr files and script files must include the same string in their names to be matched
##! multiple asr files for one script should have a counter

##* these paths will be taken from the html panel browse dialog and fed to this python script
asr_folder = '/Users/tristangardner/Documents/Programming/3. Test Media/Wayne Mayer/ASR'
scripts_folder = '/Users/tristangardner/Documents/Programming/3. Test Media/Wayne Mayer/Processed Scripts'

file_dict = {}

for script_file in os.listdir(scripts_folder):
    script_file_name = os.path.splitext(script_file)[0]
    script_file_path = os.path.join(scripts_folder, script_file)
    file_dict[script_file_path] = []
    # print(f'ASR files for script: {script_file_name}')
    
    for asr_file in os.listdir(asr_folder):
        asr_file_name = os.path.splitext(asr_file)[0]
        asr_file_path = os.path.join(asr_folder, asr_file)
        if script_file_name in asr_file_name:
            file_dict[script_file_path].append(asr_file_path)
            # print(f'\tASR File Path: {asr_file_path}')            
    # print('\n')
    
file_dict = dict(sorted(file_dict.items()))
file_dict_df = pd.DataFrame.from_dict(file_dict, orient='index').transpose()
file_dict_df

,/Users/tristangardner/Documents/Programming/3. Test Media/Wayne Mayer/Processed Scripts/Bio.txt,/Users/tristangardner/Documents/Programming/3. Test Media/Wayne Mayer/Processed Scripts/E1A.txt,/Users/tristangardner/Documents/Programming/3. Test Media/Wayne Mayer/Processed Scripts/E1B.txt,/Users/tristangardner/Documents/Programming/3. Test Media/Wayne Mayer/Processed Scripts/E1C.txt,/Users/tristangardner/Documents/Programming/3. Test Media/Wayne Mayer/Processed Scripts/E1D.txt,/Users/tristangardner/Documents/Programming/3. Test Media/Wayne Mayer/Processed Scripts/E1E.txt,/Users/tristangardner/Documents/Programming/3. Test Media/Wayne Mayer/Processed Scripts/E1F.txt
0,/Users/tristangardner/Documents/Programming/3. Test Media/Wayne Mayer/ASR/ASR_EXO_WM_Bio_1_proxyWT_240117-2300.json,/Users/tristangardner/Documents/Programming/3. Test Media/Wayne Mayer/ASR/ASR_EXO_WM_E1A_1_proxyWT_240117-2303.json,/Users/tristangardner/Documents/Programming/3. Test Media/Wayne Mayer/ASR/ASR_EXO_WM_E1B_1_proxyWT_240117-2325.json,/Users/tristangardner/Documents/Programming/3. Test Media/Wayne Mayer/ASR/ASR_EXO_WM_E1C_1_proxyWT_240117-2344.json,None,/Users/tristangardner/Documents/Programming/3. Test Media/Wayne Mayer/ASR/ASR_EXO_WM_E1E_1_proxyWT_240117-2358.json,/Users/tristangardner/Documents/Programming/3. Test Media/Wayne Mayer/ASR/ASR_EXO_WM_E1F_1_proxyWT_240118-2353.json
1,/Users/tristangardner/Documents/Programming/3. Test Media/Wayne Mayer/ASR/ASR_EXO_WM_Bio_2_proxyWT_240117-2259.json,None,None,/Users/tristangardner/Documents/Programming/3. Test Media/Wayne Mayer/ASR/ASR_EXO_WM_E1C_2_proxyWT_240117-2342.json,None,None,None
2,/Users/tristangardner/Documents/Programming/3. Test Media/Wayne Mayer/ASR/ASR_EXO_WM_Bio_3_proxyWT_240117-2300.json,None,None,None,None,None,None


In [17]:
def createTakes(script_sentences, asr_file_path, debug=False):
    ##* returns a dictionary of cuts, each cell has objects converted to strings for JavaScript ingest

    print_and_display = debug

    ## CORPUS CREATION
    #! each script must begin with the title of the script in one line, and a " in the next line - the script should start on the next line - it should have another " after the last script line

    with open (asr_file_path, 'r') as file:
        asr_dict = json.load(file)

    # Initialize variables for sentence assembly
    asr_sentences           = []
    sentence_start_times    = []
    sentence_end_times      = []
    current_sentence        = ''
    current_start_time      = None

    # Iterate through segments
    for segment in asr_dict['segments']:
        words = segment['words']

        for word in words:
            word_text = word['text']
            word_start_time = word['start']
            word_end_time = word['end']

            if not current_sentence:
                # Start a new sentence
                current_sentence = word_text
                current_start_time = word_start_time
            else:
                # Append word to current sentence
                current_sentence += ' ' + word_text

            # Check if the word ends with punctuation
            if any(punct in word_text for punct in ['.', '?', '!']):
                # End of the current sentence
                asr_sentences.append(current_sentence)
                sentence_start_times.append(current_start_time)
                sentence_end_times.append(word_end_time)

                # Reset for the next sentence
                current_sentence = ''
                current_start_time = None

    # Handle case where the last word does not end with punctuation
    if current_sentence:
        asr_sentences.append(current_sentence)
        sentence_end_times.append(segment['end'])

    # Combine sentences with their start and end times
    assembled_sentences     = [{"Sentence": sentence, "Start Time": start, "End Time": end}
                        for sentence, start, end in zip(asr_sentences, sentence_start_times, sentence_end_times)]

    df_assembled_sentences  = pd.DataFrame(assembled_sentences)
    # df_assembled_sentences

    # Find the maximum length among the lists
    max_length              = max(len(asr_sentences), len(sentence_start_times), len(sentence_end_times), len(script_sentences)) ##! fix script sentences first to fill itself from script file 
    script_sentences.extend([None] * (max_length - len(script_sentences)))
    asr_sentences.extend([None] * (max_length - len(asr_sentences)))
    sentence_start_times.extend([None] * (max_length - len(sentence_start_times)))
    sentence_end_times.extend([None] * (max_length - len(sentence_end_times)))

    """ 
    asr_segments.extend([None] * (max_length - len(asr_segments)))
    segment_start_times.extend([None] * (max_length - len(segment_start_times)))
    segment_end_times.extend([None] * (max_length - len(segment_end_times))) """

    dict_corpus = {
        'Script Sentences': script_sentences,
        'Sentence': asr_sentences, 
        'Sentence Start': sentence_start_times, 
        'Sentence End': sentence_end_times,
    }

    df_corpus = pd.DataFrame(dict_corpus, columns=['Script Sentences', 'Sentence', 'Sentence Start', 'Sentence End'])
    if print_and_display == True:
        print('\n********** Corpus DF **********\n')
        display(df_corpus)

    """ ## SENTENCE SIMILARITY UPSTREAM """
    results = {
        'Matches': {
        'Script Sentence Index': [],
        'Script Sentence': [],
        'Take Index': [], # ASR Segment Index
        'ASR Matches': [],
        'Similarity Score': [],
        'Start Time': [],
        'End Time': []
        },
        'Unmatched': {}
    }

    # region: Generate Embeddings
    transformer = 'sentence-transformers/paraphrase-multilingual-mpnet-base-v2'
    model = SentenceTransformer(transformer)

    sentences1 = [sentence for sentence in script_sentences if sentence is not None]
    sentences2 = [sentence for sentence in asr_sentences if sentence is not None]

    embeddings1 = model.encode(sentences1, convert_to_tensor=True)
    embeddings2 = model.encode(sentences2, convert_to_tensor=True)
    # endregion: Generate Embeddings

    for i, embedding1 in enumerate(embeddings1):
        # Compute cosine similarities
        cosine_scores = util.cos_sim(embedding1, embeddings2)
        cosine_scores = cosine_scores.flatten()

        # Clamp the cosine scores above threshold
        valid_indices = np.where(cosine_scores > 0.65)[0]
        valid_scores = cosine_scores[valid_indices]

        if len(valid_scores) > 0:
            percentile = np.percentile(valid_scores, 90) # this doesn't do much if multiple takes have the same exact words --> group window weight is needed

            # Find indices where the cosine score is equal to or higher than the 70th percentile
            top_indices = valid_indices[np.where(valid_scores >= percentile)[0]]
            top_scores = cosine_scores[top_indices]

            # Append the results to your dictionary
            for index, score in zip(top_indices, top_scores):
                match_data = (i, sentences1[i], index, sentences2[index], score.item(), sentence_start_times[index], sentence_end_times[index])
                for key, value in zip(results['Matches'].keys(), match_data):
                    results['Matches'][key].append(value)
        else:
            # Handle case with no valid matches
            results['Matches']['Script Sentence Index'].append(i)
            results['Matches']['Script Sentence'].append(sentences1[i])
            results['Matches']['Take Index'].append(None)
            results['Matches']['ASR Matches'].append("No Matches Found")
            results['Matches']['Similarity Score'].append(None)
            results['Matches']['Start Time'].append(None)
            results['Matches']['End Time'].append(None)

    df_results = pd.DataFrame(results['Matches'])
    if print_and_display == True:
        print('\n********** Results DF **********\n')
        display(df_results)

    """ ## Sort by Start Time """
    df_results_1_sortStart = df_results.copy()
    df_results_1_sortStart = df_results_1_sortStart.sort_values(by=['Start Time'])
    df_results_1_sortStart.reset_index(drop=True)
    """## Grouping Sentences into Time Windows  """
    df_results_2_grouped = df_results_1_sortStart.copy()

    # Initialize a list to keep track of each group's time window
    group_windows = []

    for i in range(len(df_results_2_grouped)):
        start_time = df_results_2_grouped.at[i, 'Start Time']
        if start_time == None:
            continue
        end_time = df_results_2_grouped.at[i, 'End Time']
        assigned = False

        # Check if the sentence fits into the time window of any existing group
        for group_id, (group_start, group_end) in enumerate(group_windows):
            if max(group_start, start_time) - min(group_end, end_time) <= 3.5: #! @Param: Time Window
                # Update the group's time window
                group_windows[group_id] = (min(group_start, start_time), max(group_end, end_time))
                df_results_2_grouped.at[i, 'Group ID'] = chr(65 + group_id)  # Convert group_id to a letter
                assigned = True
                break

        if not assigned:
            # Start a new group and assign a letter as ID
            new_group_id = len(group_windows)
            df_results_2_grouped.at[i, 'Group ID'] = chr(65 + new_group_id)  # Convert new_group_id to a letter
            group_windows.append((start_time, end_time))

    df_results_2_grouped.reset_index(drop=True)
    ## Group Sizesx
    # Group by 'Group ID' and count the size of each group
    group_sizes = df_results_2_grouped.groupby('Group ID').size()

    # Map the group size back to the original DataFrame
    df_results_2_grouped['Group Size'] = df_results_2_grouped['Group ID'].map(group_sizes)

    # Display the first few rows of the DataFrame
    df_results_2_grouped.sort_values(by=['Start Time'])
    if print_and_display == True:
        print('\n********** Grouped DF **********\n')
        display(df_results_2_grouped)

    ## Create Final dict

    df_results_grouped_2_dropDuplicates = df_results_2_grouped.sort_values(by=['Start Time']).drop_duplicates(subset='Take Index')

    results_dict = df_results_grouped_2_dropDuplicates.dropna().to_dict('records')

    # df_results_grouped_D_dropDuplicates

    matches = {
        'All Takes': {}, ## DONE 
        'Longest Takes': {}, # Each asr match for every script sentence index that has the longest associated group size
        'Highest Score Takes': {},
        'Last Takes': {}, # the last-most take of each script sentence
        'Script Sentence Takes': {} # all matches for each script sentence 
        #'Add Libs': {} # just the ones that are inside of a long take?
    }

    matches['All Takes'] = results_dict

    ## Filling add Group Start and Group End to each dictionary in 'All Takes'
    # Add 'Group Start' and 'Group End' to each dictionary in 'All Takes'
    for take in matches['All Takes']:
        # Find the group that this take belongs to
        group = df_results_grouped_2_dropDuplicates[df_results_grouped_2_dropDuplicates['Group ID'] == take['Group ID']]
        # Find the 'Start Time' of the take with the lowest start time in the group
        group_start = group['Start Time'].min()
        # Find the 'End Time' of the take with the highest end time in the group
        group_end = group['End Time'].max()
        # Add 'Group Start' and 'Group End' to the dictionary
        take['Group Start'] = group_start
        take['Group End'] = group_end

    ## Filling in Longest Takes with each script sentence match that is associated with the biggest group of which that take is a part
    # Convert 'All Takes' to a DataFrame
    df_all_takes = pd.DataFrame(matches['All Takes'])
    # Group by 'Script Sentence Index' and find the dictionary with the highest group size in each group
    df_longest_takes = df_all_takes.loc[df_all_takes.groupby('Script Sentence Index')['Group Size'].idxmax()]
    # Convert 'Longest Takes' back to a list of dictionaries
    matches['Longest Takes'] = df_longest_takes.to_dict('records')

    ## Filling highest score takes
    df_highest_score_takes = df_all_takes.loc[df_all_takes.groupby('Script Sentence Index')['Similarity Score'].idxmax()]
    matches['Highest Score Takes'] = df_highest_score_takes.to_dict('records')

    ## Filling latest recorded takes
    df_latest_recorded_takes = df_all_takes.loc[df_all_takes.groupby('Script Sentence Index')['End Time'].idxmax()]
    matches['Last Takes'] = df_latest_recorded_takes.to_dict('records')

    ## Filling script sentence matches
    df_script_sentence_matches = df_all_takes.sort_values(by=['Script Sentence Index'])
    matches['Script Sentence Takes'] = df_script_sentence_matches.to_dict('records')
    if print_and_display == True:
        print('\n********** Script Sentence Matches DF **********\n')
        display(df_script_sentence_matches)

    ## Create Final cuts dict & JSON
    cuts = {}

    cuts['Longest Takes'] = getCuts(matches['Longest Takes'])
    cuts['Highest Score Takes'] = getCuts(matches['Highest Score Takes'])
    cuts['Last Takes'] = getCuts(matches['Last Takes'])
    cuts['Script Sentence Takes'] = getCutsByGroupAndSentence(matches['Script Sentence Takes'])

    # cuts_df = pd.DataFrame(cuts, columns=['Longest Takes', 'Highest Score Takes', 'Last Takes', 'Script Sentence Takes'])
    # if print_and_display == True:
    #     print('\n********** Cuts DF **********\n')
    #     display(cuts_df)

    ## Convert Keys/Values in Cuts Dict so so JSON.dump ingests all correct object types
    def convert_numpy_int_to_python_int(obj):
        if isinstance(obj, np.int64):
            return int(obj)
        elif isinstance(obj, list):
            return [convert_numpy_int_to_python_int(item) for item in obj]
        elif isinstance(obj, tuple):
            return tuple(convert_numpy_int_to_python_int(item) for item in obj)
        elif isinstance(obj, dict):
            return {key: convert_numpy_int_to_python_int(value) for key, value in obj.items()}
        return obj

    # Apply this function to your dictionary
    converted_cuts_dict = convert_numpy_int_to_python_int(cuts)

    return converted_cuts_dict

In [54]:
""" ## Main Folder Run """

cuts_export_folder = '/Users/tristangardner/Documents/Programming/3. Test Media/Wayne Mayer/Cuts'

for column in file_dict_df:
    script_file_path = column
    with open(script_file_path, 'r') as file:
        script_sentences = file.read().splitlines()
        script_sentences = script_sentences[2:-1] ##* where script_sentences is finalized for use
        
    for index, value in file_dict_df[column].items():
        if pd.notna(value):  # Check if the value is not NaN (None in your case)
            asr_file_path = value
            converted_cuts_dict = createTakes(script_sentences, asr_file_path)
            cuts_file_script = os.path.splitext(os.path.basename(script_file_path))[0]
            cuts_file_name = f'{cuts_file_script}_{index+1}'
            toJson(converted_cuts_dict, cuts_file_name, prefix='cuts')


In [18]:
""" ## Main Single File Run """
asr_E1D = '/Users/tristangardner/Documents/Programming/3. Test Media/Wayne Mayer/ASR/ASR_EXO_WM_E1D_1_proxyWT_240128-2114.json'
asr_E1E = '/Users/tristangardner/Documents/Programming/3. Test Media/Wayne Mayer/ASR/ASR_EXO_WM_E1E_1_proxyWT_240117-2358.json'

asr_file_path = asr_E1E
script_file_path = '/Users/tristangardner/Documents/Programming/3. Test Media/Wayne Mayer/Processed Scripts/E1E.txt'

with open(script_file_path, 'r') as file:
    script_sentences = file.read().splitlines()
    script_sentences = script_sentences[2:-1] ##* where script_sentences is finalized for use

converted_cuts_dict = createTakes(script_sentences, asr_file_path)
cuts_file_script = os.path.splitext(os.path.basename(script_file_path))[0]
cuts_file_name = cuts_file_script
toJson(converted_cuts_dict, cuts_file_name, prefix='cuts')


In [8]:
# converting a cuts file to csv just for longest takes, which I don't really need to be doing
from jsonToCSV import jsonToCSV

file_path = '/Users/tristangardner/Documents/Programming/3. Test Media/Wayne Mayer/Cuts/cuts_E1D_1_240128-2143.json'
key = 'Longest Takes'
jsonToCSV(file_path, key, '_Longest Takes')